In [1]:
#@title Install requirements. { display-mode: "form" }
# Install requirements
#!pip install torch transformers sentencepiece h5py

In [2]:
#@title Set up working directories and download files/checkpoints. { display-mode: "form" }
# Create directory for storing model weights (2.3GB) and example sequences.
# Here we use the encoder-part of ProtT5-XL-U50 in half-precision (fp16) as 
# it performed best in our benchmarks (also outperforming ProtBERT-BFD).
# Also download secondary structure prediction checkpoint to show annotation extraction from embeddings
!mkdir protT5 # root directory for storing checkpoints, results etc
!mkdir protT5/protT5_checkpoint # directory holding the ProtT5 checkpoint
!mkdir protT5/sec_struct_checkpoint # directory storing the supervised classifier's checkpoint
!mkdir protT5/output # directory for storing your embeddings & predictions
!wget -nc -P protT5/ https://rostlab.org/~deepppi/example_seqs.fasta
# Huge kudos to the bio_embeddings team here! We will integrate the new encoder, half-prec ProtT5 checkpoint soon
!wget -nc -P protT5/sec_struct_checkpoint http://data.bioembeddings.com/public/embeddings/feature_models/t5/secstruct_checkpoint.pt

A subdirectory or file protT5 already exists.
Error occurred while processing: protT5.
A subdirectory or file # already exists.
Error occurred while processing: #.
A subdirectory or file root already exists.
Error occurred while processing: root.
A subdirectory or file directory already exists.
Error occurred while processing: directory.
A subdirectory or file for already exists.
Error occurred while processing: for.
A subdirectory or file storing already exists.
Error occurred while processing: storing.
A subdirectory or file checkpoints already exists.
Error occurred while processing: checkpoints.
A subdirectory or file results already exists.
Error occurred while processing: results.
A subdirectory or file etc already exists.
Error occurred while processing: etc.
The syntax of the command is incorrect.
The syntax of the command is incorrect.
The syntax of the command is incorrect.
'predictions' is not recognized as an internal or external command,
operable program or batch file.
'wg

In [1]:
# In the following you can define your desired output. Current options:
# per_residue embeddings
# per_protein embeddings
# secondary structure predictions

# Replace this file with your own (multi-)FASTA
# Headers are expected to start with ">";
#seq_path = "protT5/Zmays_493_RefGen_V4.protein_primaryTranscriptOnly.canonical.fa"

seq_name = "fasta_file_name"
seq_path = "protT5/"+seq_name+".fa"

# whether to retrieve embeddings for each residue in a protein 
# --> Lx1024 matrix per protein with L being the protein's length
# as a rule of thumb: 1k proteins require around 1GB RAM/disk
per_residue = True 
per_residue_path = "protT5/output/"+seq_name+".residue.h5" # where to store the embeddings

# whether to retrieve per-protein embeddings 
# --> only one 1024-d vector per protein, irrespective of its length
per_protein = True
per_protein_path = "protT5/output/"+seq_name+".protein.h5" # where to store the embeddings

# whether to retrieve secondary structure predictions
# This can be replaced by your method after being trained on ProtT5 embeddings
sec_struct = True
sec_struct_path = "protT5/output/"+seq_name+".struct.fa" # file for storing predictions

# make sure that either per-residue or per-protein embeddings are stored
assert per_protein is True or per_residue is True or sec_struct is True, print(
    "Minimally, you need to active per_residue, per_protein or sec_struct. (or any combination)")


In [2]:
#@title Import dependencies and check whether GPU is available. { display-mode: "form" }
from transformers import T5EncoderModel, T5Tokenizer
import torch
import h5py
import time
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
#device = torch.device("cpu")
print("Using {}".format(device))

Using cuda:0


In [3]:
#@title Network architecture for secondary structure prediction. { display-mode: "form" }
# Convolutional neural network (two convolutional layers) to predict secondary structure
class ConvNet( torch.nn.Module ):
    def __init__( self ):
        super(ConvNet, self).__init__()
        # This is only called "elmo_feature_extractor" for historic reason
        # CNN weights are trained on ProtT5 embeddings
        self.elmo_feature_extractor = torch.nn.Sequential(
                        torch.nn.Conv2d( 1024, 32, kernel_size=(7,1), padding=(3,0) ), # 7x32
                        torch.nn.ReLU(),
                        torch.nn.Dropout( 0.25 ),
                        )
        n_final_in = 32
        self.dssp3_classifier = torch.nn.Sequential(
                        torch.nn.Conv2d( n_final_in, 3, kernel_size=(7,1), padding=(3,0)) # 7
                        )
        
        self.dssp8_classifier = torch.nn.Sequential(
                        torch.nn.Conv2d( n_final_in, 8, kernel_size=(7,1), padding=(3,0))
                        )
        self.diso_classifier = torch.nn.Sequential(
                        torch.nn.Conv2d( n_final_in, 2, kernel_size=(7,1), padding=(3,0))
                        )
        

    def forward( self, x):
        # IN: X = (B x L x F); OUT: (B x F x L, 1)
        x = x.permute(0,2,1).unsqueeze(dim=-1) 
        x         = self.elmo_feature_extractor(x) # OUT: (B x 32 x L x 1)
        d3_Yhat   = self.dssp3_classifier( x ).squeeze(dim=-1).permute(0,2,1) # OUT: (B x L x 3)
        d8_Yhat   = self.dssp8_classifier( x ).squeeze(dim=-1).permute(0,2,1) # OUT: (B x L x 8)
        diso_Yhat = self.diso_classifier(  x ).squeeze(dim=-1).permute(0,2,1) # OUT: (B x L x 2)
        return d3_Yhat, d8_Yhat, diso_Yhat

In [4]:
#@title Load the checkpoint for secondary structure prediction. { display-mode: "form" }
def load_sec_struct_model():
  checkpoint_dir="./protT5/sec_struct_checkpoint/secstruct_checkpoint.pt"
  state = torch.load( checkpoint_dir )
  model = ConvNet()
  model.load_state_dict(state['state_dict'])
  model = model.eval()

  # https://github.com/agemagician/ProtTrans/blob/cc432db8ce0b28754a4383f53b0a387267574f1c/Embedding/PyTorch/Use_Case:NetSurfp_Dataset_Secondary_Structure_Prediction_(3_States)_Feature_Extraction.ipynb
  if torch.cuda.is_available():
    model = model.half()

  model = model.to(device)
  # I'm not sure if this information about the epoch is important
  #print('Loaded sec. struct. model from epoch: {:.1f}'.format(state['epoch']))

  

  return model

In [5]:
#@title Load encoder-part of ProtT5 in half-precision. { display-mode: "form" }
# Load ProtT5 in half-precision (more specifically: the encoder-part of ProtT5-XL-U50) 
def get_T5_model():
    model = T5EncoderModel.from_pretrained("Rostlab/prot_t5_xl_half_uniref50-enc")
    model = model.to(device) # move model to GPU
    model = model.eval() # set model to evaluation model
    
    # https://github.com/agemagician/ProtTrans/blob/cc432db8ce0b28754a4383f53b0a387267574f1c/Embedding/PyTorch/Use_Case:NetSurfp_Dataset_Secondary_Structure_Prediction_(3_States)_Feature_Extraction.ipynb
    if torch.cuda.is_available():
        model = model.half()
    
    tokenizer = T5Tokenizer.from_pretrained('Rostlab/prot_t5_xl_half_uniref50-enc', do_lower_case=False)

    return model, tokenizer

In [6]:
#@title Read in file in fasta format. { display-mode: "form" }
def read_fasta( fasta_path, split_char="!", id_field=0):
    '''
        Reads in fasta file containing multiple sequences.
        Split_char and id_field allow to control identifier extraction from header.
        E.g.: set split_char="|" and id_field=1 for SwissProt/UniProt Headers.
        Returns dictionary holding multiple sequences or only single 
        sequence, depending on input file.
    '''
    
    seqs = dict()
    with open( fasta_path, 'r' ) as fasta_f:
        for line in fasta_f:
            # get uniprot ID from header and create new entry
            if line.startswith('>'):
                uniprot_id = line.replace('>', '').strip().split(split_char)[id_field]
                # replace tokens that are mis-interpreted when loading h5
                uniprot_id = uniprot_id.replace("/","_").replace(".","_")
                seqs[ uniprot_id ] = ''
            else:
                # repl. all whie-space chars and join seqs spanning multiple lines, drop gaps and cast to upper-case
                seq= ''.join( line.split() ).upper().replace("-","")
                # repl. all non-standard AAs and map them to unknown/X
                seq = seq.replace('U','X').replace('Z','X').replace('O','X')
                seqs[ uniprot_id ] += seq 
    example_id=next(iter(seqs))
    print("Read {} sequences.".format(len(seqs)))
    print("Example:\n{}\n{}".format(example_id,seqs[example_id]))

    return seqs

In [7]:
#@title Generate embeddings. { display-mode: "form" }
# Generate embeddings via batch-processing
# per_residue indicates that embeddings for each residue in a protein should be returned.
# per_protein indicates that embeddings for a whole protein should be returned (average-pooling)
# max_residues gives the upper limit of residues within one batch
# max_seq_len gives the upper sequences length for applying batch-processing
# max_batch gives the upper number of sequences per batch
def get_embeddings( model, tokenizer, seqs, per_residue, per_protein, sec_struct, 
                   max_residues=4000, max_seq_len=1000, max_batch=100 ):

    if sec_struct:
      sec_struct_model = load_sec_struct_model()

    results = {"residue_embs" : dict(), 
               "protein_embs" : dict(),
               "sec_structs" : dict() 
               }

    # sort sequences according to length (reduces unnecessary padding --> speeds up embedding)
    seq_dict   = sorted( seqs.items(), key=lambda kv: len( seqs[kv[0]] ), reverse=True )
    start = time.time()
    batch = list()
    
    for seq_idx, (pdb_id, seq) in enumerate(seq_dict,1):
                
        seq = seq
        seq_len = len(seq)
        seq = ' '.join(list(seq))
        batch.append((pdb_id,seq,seq_len))

        # count residues in current batch and add the last sequence length to
        # avoid that batches with (n_res_batch > max_residues) get processed 
        n_res_batch = sum([ s_len for  _, _, s_len in batch ]) + seq_len 
        if len(batch) >= max_batch or n_res_batch>=max_residues or seq_idx==len(seq_dict) or seq_len>max_seq_len:
            pdb_ids, seqs, seq_lens = zip(*batch)
            batch = list()

            # add_special_tokens adds extra token at the end of each sequence
            token_encoding = tokenizer.batch_encode_plus(seqs, add_special_tokens=True, padding="longest")
            input_ids      = torch.tensor(token_encoding['input_ids']).to(device)
            attention_mask = torch.tensor(token_encoding['attention_mask']).to(device)
            
            try:
                with torch.no_grad():
                    # returns: ( batch-size x max_seq_len_in_minibatch x embedding_dim )
                    embedding_repr = model(input_ids, attention_mask=attention_mask)
            except RuntimeError:
                print("RuntimeError during embedding for {} (L={})".format(pdb_id, seq_len))
                continue

            if sec_struct: # in case you want to predict secondary structure from embeddings
              d3_Yhat, d8_Yhat, diso_Yhat = sec_struct_model(embedding_repr.last_hidden_state)


            for batch_idx, identifier in enumerate(pdb_ids): # for each protein in the current mini-batch
                s_len = seq_lens[batch_idx]
                # slice off padding --> batch-size x seq_len x embedding_dim  
                emb = embedding_repr.last_hidden_state[batch_idx,:s_len]
                if sec_struct: # get classification results
                    results["sec_structs"][identifier] = torch.max( d3_Yhat[batch_idx,:s_len], dim=1 )[1].detach().cpu().numpy().squeeze()
                if per_residue: # store per-residue embeddings (Lx1024)
                    results["residue_embs"][ identifier ] = emb.detach().cpu().numpy().squeeze()
                if per_protein: # apply average-pooling to derive per-protein embeddings (1024-d)
                    protein_emb = emb.mean(dim=0)
                    results["protein_embs"][identifier] = protein_emb.detach().cpu().numpy().squeeze()


    passed_time=time.time()-start
    avg_time = passed_time/len(results["residue_embs"]) if per_residue else passed_time/len(results["protein_embs"])
    print('\n############# EMBEDDING STATS #############')
    print('Total number of per-residue embeddings: {}'.format(len(results["residue_embs"])))
    print('Total number of per-protein embeddings: {}'.format(len(results["protein_embs"])))
    print("Time for generating embeddings: {:.1f}[m] ({:.3f}[s/protein])".format(
        passed_time/60, avg_time ))
    print('\n############# END #############')
    return results

In [8]:
#@title Write embeddings to disk. { display-mode: "form" }
# I replace the "w" with "a" so I can append embeddings with a for-loop
def save_embeddings(emb_dict,out_path):
    with h5py.File(str(out_path), "a") as hf:
        for sequence_id, embedding in emb_dict.items():
            # noinspection PyUnboundLocalVariable
            hf.create_dataset(sequence_id, data=embedding)
    return None

In [9]:
#@title Write predictions to disk. { display-mode: "form" }
def write_prediction_fasta(predictions, out_path):
  class_mapping = {0:"H",1:"E",2:"L"} 
  with open(out_path, 'w+') as out_f:
      out_f.write( '\n'.join( 
          [ ">{}\n{}".format( 
              seq_id, ''.join( [class_mapping[j] for j in yhat] )) 
          for seq_id, yhat in predictions.items()
          ] 
            ) )
  return None

In [10]:
# Load fasta.
seqs = read_fasta(seq_path)

# Load the encoder part of ProtT5-XL-U50 in half-precision (recommended)
model, tokenizer = get_T5_model()

Read 39498 sequences.
Example:
Zm00001d000434
MALSIIPSTSSGEHAAASASATATTTTKLGQLNAAVERSWVGRRFRLAARGTTFTTELRAGTATFLTMAYILAVNASILSDSGATCTVDDCDVPSPGCKFPPVDPGYAACVARVRRDLIVATAASSVIGSFIMGAFANLPIALAPGMGTNAYFAYTVVGFHGSGTLPYRTALAAVFLEGLIFLFISIVGLRSKLAQFIPTPVRISASAGIGLFLAFIGLQSNEGVGLVGFSSSTLVTLGACPASQRASVAPVLTFPNGTVALMPGGTVSGGILCLSGRMTSPTFWLAVVGFLIIAFCLIKRVKGALIYGILFVTFVSWPRHTAVTAFPDTPAGDDSFHYFKKVFDVHRIRSTAGALDFSGIGQGYFWEALFTFLYVDILDTTGSLYTMARFAGFVDDATGEFEGQYFAFMSDATAIVFGSLLGTSPVTAFIESSTGIREGGRTGLTALTAAIYFAAALFITPVLASIPSWAVGPPLVLVGVMMMRAVAEVDWDDMRQAVPAFLTLALMPLTYSIAYGLIGGIASYMLLHSWDWACQATRRLGCLKKGGGGADSTNGGVAEQSVEEQRKDMESA


In [ ]:
# https://stackoverflow.com/questions/22878743/how-to-split-dictionary-into-multiple-dictionaries-fast

from itertools import islice

def chunks(data, SIZE):
    it = iter(data)
    for i in range(0, len(data), SIZE):
        yield {k:data[k] for k in islice(it, SIZE)}

protein_embeddings = {}
structure_predicts = {}


slice_size = 2000
count = 0
for seqs_slice in chunks(seqs, slice_size):
  # Load the encoder part of ProtT5-XL-U50 in half-precision (recommended)
  count += 1
  print("Processing:", count, "out of", str((len(seqs) // slice_size) + 1))

  # Compute embeddings and/or secondary structure predictions
  results = get_embeddings(model, tokenizer, seqs_slice,
                          per_residue, per_protein, sec_struct)
  
  # merge protein embedding dictionaries
  protein_embeddings = {**protein_embeddings, **results["protein_embs"]}
  
  # merge structure prediction fasta dictionaries
  structure_predicts = {**structure_predicts, **results["sec_structs"]}

  # save residue embeddings
  per_residue_path = per_residue_path = "protT5/output/"+seq_name+".residue.h5" # where to store the embeddings

  # Store per-residue embeddings
  if per_residue:
    save_embeddings(results["residue_embs"], per_residue_path)
    #files.download(per_residue_path) 

if per_protein:
  save_embeddings(protein_embeddings, per_protein_path)
  #files.download(per_protein_path) 

if sec_struct:
  write_prediction_fasta(structure_predicts, sec_struct_path)

Processing: 1 out of 20
RuntimeError during embedding for Zm00001d022338 (L=3882)
